Text Analytics Demo: https://aidemos.microsoft.com/text-analytics

Pricing: https://azure.microsoft.com/en-gb/pricing/details/cognitive-services/text-analytics/#pricing

Clean Up Azure Resources:

In [24]:
import os
os.path.abspath(os.getcwd())

In [2]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name),encoding="utf8").read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

Review1.txt
Good Hotel and staff
The Royal Hotel, London, UK
3/2/2018
Clean rooms, good service, great location near Buckingham Palace and Westminster Abbey, and so on. We thoroughly enjoyed our stay. The courtyard is very peaceful and we went to a restaurant which is part of the same group and is Indian ( West coast so plenty of fish) with a Michelin Star. We had the taster menu which was fabulous. The rooms were very well appointed with a kitchen, lounge, bedroom and enormous bathroom. Thoroughly recommended.

Review2.txt
Tired hotel with poor service
The Royal Hotel, London, United Kingdom
5/6/2018
This is a old hotel (has been around since 1950's) and the room furnishings are average - becoming a bit old now and require changing. The internet didn't work and had to come to one of their office rooms to check in for my flight home. The website says it's close to the British Museum, but it's too far to walk.

Review3.txt
Tired hotel with poor service
The Royal Hotel, London, United Ki

In [23]:
cog_key = 'your key'
cog_endpoint = 'End point link'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ready to use cognitive services at End point link using key your key


In [4]:
!pip install azure-ai-textanalytics
!pip install azure-cognitiveservices-language-textanalytics

# Language Detection

In [5]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name
    

Review1.txt
 - Language: English
 - Code: en
 - Score: 1.0

Review2.txt
 - Language: English
 - Code: en
 - Score: 1.0

Review3.txt
 - Language: English
 - Code: en
 - Score: 1.0

Review4.txt
 - Language: English
 - Code: en
 - Score: 1.0

Review5.txt
 - Language: English
 - Code: en
 - Score: 1.0



In [6]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(cog_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=cog_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [7]:
def language_detection_example(client):
    try:
        documents = ["சிறப்பு எழுத்துக்குறிகள். இதை மேம்படுத்த உதவவும்! Google உள்ளீட்டு கருவியைப் பெறுக "]
        response = client.detect_language(documents = documents, country_hint = 'us')[0]
        print("Language: ", response.primary_language.name)

    except Exception as err:
        print("Encountered exception. {}".format(err))
language_detection_example(client)

Language:  Tamil


# Detect Personally Identifiable Information (PII)

https://docs.microsoft.com/en-us/azure/cognitive-services/language-service/personally-identifiable-information/concepts/entity-categories

In [9]:
#key = "paste-your-key-here"
#endpoint = "paste-your-endpoint-here"


# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client):
    documents = [
        "Ingram Micro India SSC Private Limited - Chennai Central Square, I, Guindy Industrial Estate, SIDCO Industrial Estate, Guindy, Chennai, Tamil Nadu 600032",
        "SKCL Central Square 1 https://www.espncricinfo.com/",
        "Ingram Micro's origins trace back to the founding of distributor Micro D, Inc. in July 1979 by husband and wife team, Geza Czige and Lorraine Mecca, who were both teachers. The company started in Southern California and in its first year of business achieved approximately $3.5 million in sales."
    ]
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))
pii_recognition_example(client)

Redacted Text: ************************************** - ***************************************************, ***********************************************************
Entity: Ingram Micro India SSC Private Limited
	Category: Organization
	Confidence Score: 0.9
	Offset: 0
	Length: 38
Entity: Chennai Central Square, I, Guindy Industrial Estate
	Category: Address
	Confidence Score: 0.92
	Offset: 41
	Length: 51
Entity: SIDCO Industrial Estate, Guindy, Chennai, Tamil Nadu 600032
	Category: Address
	Confidence Score: 0.66
	Offset: 94
	Length: 59
Redacted Text: SKCL ************** 1 *****************************
Entity: Central Square
	Category: Address
	Confidence Score: 0.71
	Offset: 5
	Length: 14
Entity: https://www.espncricinfo.com/
	Category: URL
	Confidence Score: 0.8
	Offset: 22
	Length: 29
Redacted Text: ************'s origins trace back to the founding of distributor ************* in ********* by ******* and **** team, ********** and **************, who were both ********. The comp

# Key Phrase extraction

In [10]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    print(key_phrases)
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Review1.txt

Key Phrases:
['Good Hotel', 'good service', 'Clean rooms', 'Royal Hotel', 'great location', 'Buckingham Palace', 'Westminster Abbey', 'fish', 'West coast', 'lounge', 'bedroom', 'enormous bathroom', 'group', 'kitchen', 'London', 'UK', 'taster menu', 'Michelin Star', 'staff', 'courtyard']
	 Good Hotel
	 good service
	 Clean rooms
	 Royal Hotel
	 great location
	 Buckingham Palace
	 Westminster Abbey
	 fish
	 West coast
	 lounge
	 bedroom
	 enormous bathroom
	 group
	 kitchen
	 London
	 UK
	 taster menu
	 Michelin Star
	 staff
	 courtyard


Review2.txt

Key Phrases:
['old hotel', 'Royal Hotel', 'Tired hotel', 'London', 'United Kingdom', 'room furnishings', 'poor service', 'British Museum', 'website', 'office rooms', 'flight home', 'internet']
	 old hotel
	 Royal Hotel
	 Tired hotel
	 London
	 United Kingdom
	 room furnishings
	 poor service
	 British Museum
	 website
	 office rooms
	 flight home
	 internet


Review3.txt

Key Phrases:
['old hotel', 'Royal Hotel', 'Tired hotel'

In [11]:
def key_phrase_extraction_example(client):

    try:
        documents = ["Saurashtra fast bowler Jaydev Unadkat, 30, has been a top earner at IPL auctions and a consistent wicket-taker in domestic cricket, but hasn't found a place in India A sides. The last of his 18 internationals came four years ago. Bengal batter Abhimanyu Easwaran, 26, is a regular India A player and was in the Test reserves last year. Thirty-six-year-old Faiz Fazal, batter and Vidarbha captain, has not got any India opportunities since playing an ODI against Zimbabwe in 2016."]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            print("\tKey Phrases:")
            for phrase in response.key_phrases:
                print("\t\t", phrase)
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))
        
key_phrase_extraction_example(client)

	Key Phrases:
		 regular India A player
		 Saurashtra fast bowler
		 Thirty-six-year-old Faiz Fazal
		 A sides.
		 India opportunities
		 Jaydev Unadkat
		 top earner
		 IPL auctions
		 consistent wicket-taker
		 domestic cricket
		 Abhimanyu Easwaran
		 Test reserves
		 Vidarbha captain
		 Bengal batter
		 place
		 18 internationals
		 ODI
		 Zimbabwe


https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis?tabs=version-3-1

# Sentiment Analysis

In [12]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

Review1.txt : positive (0.9999973773956299)
Review2.txt : negative (5.662441253662109e-07)
Review3.txt : negative (5.662441253662109e-07)
Review4.txt : negative (2.0623207092285156e-05)
Review5.txt : positive (0.5)


In [13]:
def sentiment_analysis_with_opinion_mining_example(client):

    documents = [
        "I am really exhausted with similar work.","I can't operate with the key","This is my best work at Test."
    ]

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        print("Document Sentiment: {}".format(document.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))
        for sentence in document.sentences:
            print("Sentence: {}".format(sentence.text))
            print("Sentence sentiment: {}".format(sentence.sentiment))
            print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                print("......'{}' target '{}'".format(target.sentiment, target.text))
                print("......Target score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))
                for assessment in mined_opinion.assessments:
                    print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                    print("......Assessment score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))
            print("\n")
        print("\n")
          
sentiment_analysis_with_opinion_mining_example(client)

Document Sentiment: negative
Overall scores: positive=0.00; neutral=0.00; negative=1.00 

Sentence: I am really exhausted with similar work.
Sentence sentiment: negative
Sentence score:
Positive=0.00
Neutral=0.00
Negative=1.00





Document Sentiment: negative
Overall scores: positive=0.01; neutral=0.16; negative=0.83 

Sentence: I can't operate with the key
Sentence sentiment: negative
Sentence score:
Positive=0.01
Neutral=0.16
Negative=0.83

......'negative' target 'key'
......Target score:
......Positive=0.09
......Negative=0.91

......'negative' assessment 'operate'
......Assessment score:
......Positive=0.09
......Negative=0.91





Document Sentiment: positive
Overall scores: positive=1.00; neutral=0.00; negative=0.00 

Sentence: This is my best work at Test.
Sentence sentiment: positive
Sentence score:
Positive=1.00
Neutral=0.00
Negative=0.00

......'positive' target 'work'
......Target score:
......Positive=1.00
......Negative=0.00

......'positive' assessment 'best'
......Asse

# Entity Extraction

In [14]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Review1.txt
 - Location: The Royal Hotel (https://en.wikipedia.org/wiki/The_Royal_Hotel)
 - Location: London (https://en.wikipedia.org/wiki/London)
 - DateTime: 3/2/2018 
 - Location: Buckingham Palace (https://en.wikipedia.org/wiki/Buckingham_Palace)
 - Location: Westminster Abbey (https://en.wikipedia.org/wiki/Westminster_Abbey)
 - Location: India (https://en.wikipedia.org/wiki/India)
 - Location: West Coast Main Line (https://en.wikipedia.org/wiki/West_Coast_Main_Line)
Review2.txt
 - Location: The Royal Hotel (https://en.wikipedia.org/wiki/The_Royal_Hotel)
 - Location: London (https://en.wikipedia.org/wiki/London)
 - Location: London 
 - Location: United Kingdom 
 - DateTime: 5/6/2018 
 - DateTime: since 1950's 
 - DateTime: now 
 - Location: British Museum (https://en.wikipedia.org/wiki/British_Museum)
Review3.txt
 - Location: The Royal Hotel (https://en.wikipedia.org/wiki/The_Royal_Hotel)
 - Location: London (https://en.wikipedia.org/wiki/London)
 - Location: London 
 - Location: 

In [15]:
def entity_recognition_example(client):

    try:
        documents = ["I love Italian food, especially Pizza. I had last week in Chennai"]
        result = client.recognize_entities(documents = documents)[0]

        print("Named Entities:\n")
        for entity in result.entities:
            print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,
                    "\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\tLength: \t", entity.length, "\tOffset: \t", entity.offset, "\n")

    except Exception as err:
        print("Encountered exception. {}".format(err))
entity_recognition_example(client)

Named Entities:

	Text: 	 Italian 	Category: 	 Skill 	SubCategory: 	 None 
	Confidence Score: 	 0.62 	Length: 	 7 	Offset: 	 7 

	Text: 	 food 	Category: 	 Product 	SubCategory: 	 None 
	Confidence Score: 	 0.46 	Length: 	 4 	Offset: 	 15 

	Text: 	 Pizza 	Category: 	 Product 	SubCategory: 	 None 
	Confidence Score: 	 0.99 	Length: 	 5 	Offset: 	 32 

	Text: 	 last week 	Category: 	 DateTime 	SubCategory: 	 DateRange 
	Confidence Score: 	 0.8 	Length: 	 9 	Offset: 	 45 

	Text: 	 Chennai 	Category: 	 Location 	SubCategory: 	 GPE 
	Confidence Score: 	 1.0 	Length: 	 7 	Offset: 	 58 



# Language Understanding (LUIS)

A natural language understanding (NLU) AI service that allows users to interact with your applications, bots and IoT devices by using natural language.

Language Detection Demo: https://aidemos.microsoft.com/luis/demo

LUIS: https://azure.microsoft.com/en-in/services/cognitive-services/language-understanding-intelligent-service/#overview


# Translating Text

https://docs.microsoft.com/en-us/azure/cognitive-services/translator/language-support

In [25]:
cog_key = 'Your Key'
cog_location = 'centralindia'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

Ready to use cognitive services in centralindia using key Your Key


In [19]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Saurashtra fast bowler Jaydev Unadkat, 30, has been a top earner at IPL auctions and a consistent wicket-taker in domestic cricket, but hasn't found a place in India A sides. The last of his 18 internationals came four years ago. Bengal batter Abhimanyu Easwaran, 26, is a regular India A player and was in the Test reserves last year. Thirty-six-year-old Faiz Fazal, batter and Vidarbha captain, has not got any India opportunities since playing an ODI against Zimbabwe in 2016."

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

Saurashtra fast bowler Jaydev Unadkat, 30, has been a top earner at IPL auctions and a consistent wicket-taker in domestic cricket, but hasn't found a place in India A sides. The last of his 18 internationals came four years ago. Bengal batter Abhimanyu Easwaran, 26, is a regular India A player and was in the Test reserves last year. Thirty-six-year-old Faiz Fazal, batter and Vidarbha captain, has not got any India opportunities since playing an ODI against Zimbabwe in 2016. -> Le joueur de bowling rapide de Saurashtra, Jaydev Unadkat, 30 ans, a été l’un des meilleurs revenus aux enchères IPL et un preneur de guichet constant dans le cricket national, mais n’a pas trouvé de place dans les équipes A de l’Inde. Le dernier de ses 18 internationaux est arrivé il y a quatre ans. Le frappeur du Bengale Abhimanyu Easwaran, 26 ans, est un joueur régulier de l’Inde A et était dans la réserve de test l’année dernière. Faiz Fazal, trente-six ans, frappeur et capitaine de Vidarbha, n’a pas eu d’op

In [20]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Hello -> Ciao


In [21]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

Hello -> 你好


In [22]:
text_to_translate = "गनेशन क्रिकेट का शौकीन था । अंग्रेजी कक्षा के बाद जब वह खेल के मैदान के लिए दौड़ता था । खेल का मैदान दूर होने की वजह उसे हमेशा देर हो जाती थी । उसके पुराने मित्रों ने उसे राजी कर लिया कि वह अगले दो दिन अपनी कक्षा में ना जाये ताकि वह महत्वपूर्ण मैच का अभ्यास कर ले । उसके प्रशिक्षक भी उसकी सहायता को सहमत हो गये कि वह कक्षा अध्यापक को बोल देंगे । लेकिन प्रशिक्षक ने उसे धोखा दिया और गनेशन को बिना पूर्व अनुमति के दो दिन अनुपस्थित रहने के लिए दण्ड दिया गया ।"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='en-US', from_lang='hi')
print('{} -> {}'.format(text_to_translate,translation))

गनेशन क्रिकेट का शौकीन था । अंग्रेजी कक्षा के बाद जब वह खेल के मैदान के लिए दौड़ता था । खेल का मैदान दूर होने की वजह उसे हमेशा देर हो जाती थी । उसके पुराने मित्रों ने उसे राजी कर लिया कि वह अगले दो दिन अपनी कक्षा में ना जाये ताकि वह महत्वपूर्ण मैच का अभ्यास कर ले । उसके प्रशिक्षक भी उसकी सहायता को सहमत हो गये कि वह कक्षा अध्यापक को बोल देंगे । लेकिन प्रशिक्षक ने उसे धोखा दिया और गनेशन को बिना पूर्व अनुमति के दो दिन अनुपस्थित रहने के लिए दण्ड दिया गया । -> Ganesan was fond of cricket. After english class when he ran to the playground. He was always late because the playground was far away. His old friends persuaded him not to go to his class for the next two days so that he could practice the important match. His instructor also agreed to his help that he would speak to the class teacher. But the instructor cheated on him and Ganation was punished for being absent for two days without prior permission.


Move Forward:

https://gentle-pebble-08cfc3110.azurestaticapps.net/#about

https://vivekraja98.medium.com/literature-text-translation-audio-synthesis-using-microsoft-azure-cognitive-services-5e35add0c79e

Reference: https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/quickstarts/client-libraries-rest-api?tabs=version-3-2&pivots=programming-language-python

https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-call-api?tabs=synchronous

#  Power BI

In [ ]:
// Returns key phrases from the text in a comma-separated list
(text) => let
    apikey      = "YOUR_API_KEY_HERE",
    endpoint    = "https://<your-custom-subdomain>.cognitiveservices.azure.com/text/analytics" & "/v3.0/keyPhrases",
    jsontext    = Text.FromBinary(Json.FromValue(Text.Start(Text.Trim(text), 5000))),
    jsonbody    = "{ documents: [ { language: ""en"", id: ""0"", text: " & jsontext & " } ] }",
    bytesbody   = Text.ToBinary(jsonbody),
    headers     = [#"Ocp-Apim-Subscription-Key" = apikey],
    bytesresp   = Web.Contents(endpoint, [Headers=headers, Content=bytesbody]),
    jsonresp    = Json.Document(bytesresp),
    keyphrases  = Text.Lower(Text.Combine(jsonresp[documents]{0}[keyPhrases], ", "))
in  keyphrases

In [ ]:
// Returns the sentiment label of the text, for example, positive, negative or mixed.
(text) => let
    apikey = "YOUR_API_KEY_HERE",
    endpoint = "<your-custom-subdomain>.cognitiveservices.azure.com" & "/text/analytics/v3.1/sentiment",
    jsontext = Text.FromBinary(Json.FromValue(Text.Start(Text.Trim(text), 5000))),
    jsonbody = "{ documents: [ { language: ""en"", id: ""0"", text: " & jsontext & " } ] }",
    bytesbody = Text.ToBinary(jsonbody),
    headers = [#"Ocp-Apim-Subscription-Key" = apikey],
    bytesresp = Web.Contents(endpoint, [Headers=headers, Content=bytesbody]),
    jsonresp = Json.Document(bytesresp),
    sentiment   = jsonresp[documents]{0}[sentiment] 
    in sentiment

In [ ]:
// Returns the two-letter language code (for example, 'en' for English) of the text
(text) => let
    apikey      = "YOUR_API_KEY_HERE",
    endpoint    = "https://<your-custom-subdomain>.cognitiveservices.azure.com" & "/text/analytics/v3.1/languages",
    jsontext    = Text.FromBinary(Json.FromValue(Text.Start(Text.Trim(text), 5000))),
    jsonbody    = "{ documents: [ { id: ""0"", text: " & jsontext & " } ] }",
    bytesbody   = Text.ToBinary(jsonbody),
    headers     = [#"Ocp-Apim-Subscription-Key" = apikey],
    bytesresp   = Web.Contents(endpoint, [Headers=headers, Content=bytesbody]),
    jsonresp    = Json.Document(bytesresp),
    language    = jsonresp [documents]{0}[detectedLanguage] [iso6391Name] in language

In [ ]:
// Returns the name (for example, 'English') of the language in which the text is written
(text) => let
    apikey      = "YOUR_API_KEY_HERE",
    endpoint    = "https://<your-custom-subdomain>.cognitiveservices.azure.com" & "/text/analytics/v3.1/languages",
    jsontext    = Text.FromBinary(Json.FromValue(Text.Start(Text.Trim(text), 5000))),
    jsonbody    = "{ documents: [ { id: ""0"", text: " & jsontext & " } ] }",
    bytesbody   = Text.ToBinary(jsonbody),
    headers     = [#"Ocp-Apim-Subscription-Key" = apikey],
    bytesresp   = Web.Contents(endpoint, [Headers=headers, Content=bytesbody]),
    jsonresp    = Json.Document(bytesresp),
    language    jsonresp [documents]{0}[detectedLanguage] [iso6391Name] in language